In [41]:
library(ca)
library(gtools)
library(mclust)
library(rDEA)

In [2]:
data("wg93")

In [3]:
head(wg93)

A,B,C,D,sex,age,edu
2,3,4,3,2,2,3
3,4,2,3,1,3,4
2,3,2,4,2,3,2
2,2,2,2,1,2,3
3,3,3,3,1,5,2
3,4,4,5,1,3,2


In [4]:
train_idx <- sample(nrow(wg93), nrow(wg93) * 0.9, replace = FALSE)

In [5]:
train <- wg93[train_idx, ]
test <- wg93[-train_idx, ]

In [24]:

.euc_dist <- function(x1, x2) {
  return (sqrt(sum((x1 - x2) ^ 2)))
}

.round_num <- function(num) round(num, 6)

.extract_cluster_data <- function(df, y, active, passives) {

  # check df for > 1 dim of each variable

  # Create CA table using the active and passive variables
  t1 <- table(df[[active]], df[[y]])
  rownames(t1) <- paste(active, rownames(t1), sep = "_")

  t <- t1

  last.sup.index <- nrow(t1) + 1
  sup.indeces <- c()
  # sup.level.counts <- c()
  for(passive in passives) {
    t.passive <- table(df[[passive]], df[[y]])
    rownames(t.passive) <- paste(passive, rownames(t.passive), sep = "_")
    sup.indeces <- c(sup.indeces, last.sup.index)
    t <- rbind(t, t.passive)
    last.sup.index <- last.sup.index + nrow(t.passive)
  }

  fit <- ca(t, suprow = c((nrow(t1) + 1):nrow(t)), nd=nrow(t))

  sup.rows <- fit$rowsup
  active.rows <- setdiff(c(1:nrow(t)), sup.rows)

  # print(sup.level.counts)

  passive_levels <- list()
  i <- 1
  for(p in passives) {
    # print(length(levels(wg93[[p]])))
    passive_levels[[i]] <- c(1:length(levels(df[[p]])))
    i <- i + 1
  }

  passive.perms <- expand.grid(passive_levels)

  # measure the distance from each level of the predicted variable to each passive/activate variable level

  rows <- c()
  coord.names <- c()
  levels <- c()
  for(i in 1:nrow(fit$colcoord)) {
    for(j in active.rows) {

      active.col.coords <- fit$colcoord[i, ]
      active.col.level <- colnames(t)[i]
      active.row.coords <- fit$rowcoord[j, ]
      active.row.level <- rownames(t)[j]
      x <- .euc_dist(active.col.coords, active.row.coords)

      for(k in 1:nrow(passive.perms)) {
        sup.row.dists <- c()
        sup.row.names <- c()
        sup.row.levels <- c()
        for(p in 1:ncol(passive.perms)) {
          name <- passive.perms[k, p]
          sup.row.idx <- sup.indeces[p] + name - 1
          sup.row.coords <- fit$rowcoord[sup.row.idx, ]
          sup.row.dist <- .euc_dist(active.col.coords, sup.row.coords)
          sup.row.dists <- c(sup.row.dists, sup.row.dist)
          sup.row.names <- c(sup.row.names, name)
          sup.row.levels <- c(sup.row.levels, rownames(t)[sup.row.idx])
        }
        coord.names <- rbind(coord.names, paste(i, j, paste(sup.row.names, collapse = "_"), sep = "_"))
        mean.dist <- mean(c(x, sup.row.dists))
        rows <- rbind(rows, c(x, sup.row.dists, mean.dist))
        levels <- rbind(levels, c(active.col.level, active.row.level, sup.row.levels))
      }
    }
  }
  rownames(rows) <- coord.names
  rownames(levels) <- coord.names

  return(list(data=rows, levels=levels, ca=fit, data.table=t))
}

In [7]:
.cluster_data <- function(data) {
  clustered_data <- Mclust(as.matrix(data), G=1:20)
  return(list(n_cluster=clustered_data$G, classification=clustered_data$classification))
}

In [8]:
.should_use_passive_variable <- function(fit, start, level_count,
                                              first_active_coordinates,
                                              second_active_coordinates, lambda) {

  has_close_level <- function(x, y, lambda) {
    for(i in 1:nrow(x)) {
      if(.euc_dist(x[i , ], y) < lambda) {
        return(TRUE)
      }
    }
    return(FALSE)
  }

  for(level in 1:level_count) {
    level_coordinates <- fit$rowcoord[start + level, ]

    close_to_first <- has_close_level(first_active_coordinates, level_coordinates, lambda)

    if(close_to_first) {
      close_to_second <- has_close_level(second_active_coordinates, level_coordinates, lambda)

      if(close_to_second) {
        return(TRUE)
      }
    }
  }
  return(FALSE)
}

.choose_passive_variables <- function(df, y, second_active, lambda) {
  candidates <- colnames(df)
  passives <- candidates[which(candidates != y & candidates != second_active)]
  #
  t1 <- table(df[[second_active]], df[[y]])
  t <- t1
  for (p in passives) {
    t <- rbind(t, table(df[[p]], df[[y]]))
  }
  fit <- ca(t, suprow = c((nrow(t1) + 1):nrow(t)))
  # plot.ca(fit, mass = c(TRUE, TRUE), contrib = "absolute", map = "rowgreen")

  first_active_coordinates <- fit$colcoord
  second_active_coordinates <- fit$rowcoord[1:nrow(t1),]

  start <- nrow(t1)
  passives_to_use <- c()
  for(p in passives) {
    level_count <- length(levels(df[[p]]))
    if(.should_use_passive_variable(fit, start, level_count,
                                    first_active_coordinates,
                                    second_active_coordinates, lambda)) {
      passives_to_use <- c(passives_to_use, p)
    }
    start <- start + level_count
  }

  return(passives_to_use)
}

.choose_active_variables <- function(df, y, lambda) {

  actives <- c()
  should_choose_active <- function(fit) {
    for(i in 1:nrow(fit$rowcoord)) {
      for(j in 1:nrow(fit$colcoord)) {
        if(.euc_dist(fit$rowcoord[i, ], fit$colcoord[j, ]) < lambda) {
          return(TRUE)
        }
      }
    }
    return(FALSE)
  }

  df_columns <- colnames(df)
  candidates <- df_columns[which(df_columns != y)]
  for(candidate in candidates) {
    fit <- ca(table(df[[y]], df[[candidate]]))
    if(should_choose_active(fit)) {
      actives <- c(actives, candidate)
    }
  }
  return(actives)
}

In [1]:
.apply_dea <- function(cluster.fit, cluster.data, cluster.levels, output.file = "cluster_table.csv", lambda = 1.7) {

  cluster.df <- data.frame()
  mean.dist.col <- ncol(cluster.data)
  passive.dist.cols <- c(2:(mean.dist.col - 1))
  for(cluster in 1:cluster.fit$n_cluster) {
    cluster.points <- cluster.data[cluster.fit$classification == cluster,]
    levels <- cluster.levels[rownames(cluster.points), ]
    dist1 <- cluster.points[, 1]
    dist1.indicator <- sapply(dist1, function(x) {if(x < lambda) 1 else 0})
    dist.indicator <- dist1.indicator

    if(nrow(cluster.points) > 0) {
      new.df <- data.frame(
        cluster = cluster,
        name = rownames(cluster.points),
        dist1 = .round_num(dist1),
        dist1.indicator = dist1.indicator,
        dist.mean = .round_num(cluster.points[, mean.dist.col]),
        cluster.mean = .round_num(mean(colMeans(cluster.points))),
        cluster.dist1.var = .round_num(var(cluster.points[, 1])),
        active1.level = levels[, 1],
        active2.level = levels[, 2]
      )

      for(c in passive.dist.cols) {
        p.dist <- cluster.points[, c]
        p.dist.indicator <- sapply(p.dist, function(x) {if(x < lambda) 1 else 0})
        dist.indicator <- dist.indicator + p.dist.indicator
        new.df[paste("dist", c, sep = "")] <- .round_num(p.dist)
        new.df[paste("dist", c, ".indicator", sep = "")] <- p.dist.indicator
        new.df[paste("cluster.dist", c, ".var", sep = "")] <- .round_num(var(cluster.points[, c]))
        new.df[paste("passive", (c - 1), ".level", sep = "")] <- levels[, c + 1]
      }

      new.df["dist.indicator"] <- dist.indicator

      cluster.df <- rbind(cluster.df, new.df)
    }
  }

  dist.cols <- c("dist1")
  for(c in passive.dist.cols) {
    dist.cols <- c(dist.cols, paste("dist", c, sep = ""))
  }

  write.csv(cluster.df, quote = F, file = output.file, row.names = F)
  return(cluster.df)
}

In [9]:
do_impute <- function(train, test, y, lambda = 1.7) {

  second_actives <- .choose_active_variables(train, y, lambda)
  for(second_active in second_actives) {
    if(second_active != "sex") {
      passives <- .choose_passive_variables(train, y, second_active, lambda)
      if(!is.null(passives)) {
        cluster_result <- .extract_cluster_data(train, y, second_active, passives)
        cluster.data <- cluster_result$data
        cluster.levels <- cluster_result$levels
        cluster.df <- .apply_dea(.cluster_data(cluster.data[, -1]), cluster.data, cluster.levels,
                                                  output.file = paste("results/", paste("wg93_", y, "_", second_active,  ".csv", sep = ""), sep = ""),
                                                  lambda = lambda)
      }
    }
  }
}

In [10]:
lambda <- 6.0

predictions <- do_impute(train, test, y = "A", lambda=lambda)

print(sprintf("accuracy %.2f", predictions$accuracy))
print(sprintf("predicted %.2f", predictions$predicted))
print(sprintf("correct %.2f", predictions$correct))